In [20]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from sqlalchemy import create_engine
import urllib

In [77]:
base_url = "https://bank.gov.ua/ua/news/all?perPage=100&page=8"

options = Options()
options.add_argument("--headless") # Запуск без вікна
driver = webdriver.Chrome(options=options)
driver.get(base_url)
response = driver.page_source

In [78]:
soup = BeautifulSoup(response, 'html.parser')

In [79]:
p_tags = soup.find_all('p')
p_tags

[<p class="title">Національний банк України зберіг облікову ставку на рівні 15,5%</p>,
 <p class="title">Фінансові установи активніше сприяють відновленню економіки – Звіт про фінансову стабільність</p>,
 <p class="title">Національний банк відновлює окремі вимоги щодо оцінки банками кредитного ризику, підготовки планів відновлення діяльності, побудови системи управління ризиками</p>,
 <p class="title">Українці значно активніше користуються безготівковими картковими розрахунками: результати за дев’ять місяців 2025 року</p>,
 <p class="title">НБУ впроваджує черговий пакет пом'якшення валютних обмежень</p>,
 <p class="title">Міжнародні резерви зросли до 54,7 млрд дол. США за підсумками листопада</p>,
 <p class="title">НБУ присвятив нову пам’ятну монету Службі безпеки України та її унікальній спецоперації “Павутина”</p>,
 <p class="title">МВФ та українська влада досягли угоди на рівні персоналу щодо нової чотирирічної програми в межах Механізму розширеного фінансування на суму 8,1 млрд дол

In [80]:
results = []
for post in p_tags:
    # Шукаємо перше посилання <a> всередині блоку post-item
    link_tag = post.find('a')
    
    if link_tag:
        # Витягуємо URL
        href = link_tag.get('href')
        full_url = "https://bank.gov.ua" + href if href.startswith('/') else href
        
        # Витягуємо назву (текст всередині тегу <a>)
        title = link_tag.get_text(strip=True)
        
        # Додаємо в результати
        results.append({
            'title': title,
            'url': full_url
        })

In [81]:
results

[{'title': 'Виступ Голови Національного банку Андрія Пишного під час пресбрифінгу щодо рішень з монетарної політики',
  'url': 'https://bank.gov.ua/ua/news/all/vistup-golovi-natsionalnogo-banku-andriya-pishnogo-pid-chas-presbrifingu-schodo-rishen-z-monetarnoyi-politiki-20233'},
 {'title': 'Національний банк України підвищив облікову ставку до 14,5%',
  'url': 'https://bank.gov.ua/ua/news/all/natsionalniy-bank-ukrayini-pidvischiv-oblikovu-stavku-do-145'},
 {'title': 'Пресбрифінг щодо рішень Правління Національного банку з монетарної політики',
  'url': 'https://bank.gov.ua/ua/news/all/presbrifing-schodo-rishen-pravlinnya-natsionalnogo-banku-z-monetarnoyi-politiki-20223'},
 {'title': 'Порядок залучення комерційних агентів для надання фінансових платіжних послуг буде уточнено (доповнено)',
  'url': 'https://bank.gov.ua/ua/news/all/poryadok-zaluchennya-komertsiynih-agentiv-dlya-nadannya-finansovih-platijnih-poslug-bude-utochneno'},
 {'title': 'Одного небанківського надавача платіжних послу

In [82]:
print(len(results))

100


In [83]:
df = pd.DataFrame(results)
df = df.rename(columns={'url': 'link'})
df

,title,link
0,Виступ Голови Національного банку Андрія Пишно...,https://bank.gov.ua/ua/news/all/vistup-golovi-...
1,Національний банк України підвищив облікову ст...,https://bank.gov.ua/ua/news/all/natsionalniy-b...
2,Пресбрифінг щодо рішень Правління Національног...,https://bank.gov.ua/ua/news/all/presbrifing-sc...
3,Порядок залучення комерційних агентів для нада...,https://bank.gov.ua/ua/news/all/poryadok-zaluc...
4,Одного небанківського надавача платіжних послу...,https://bank.gov.ua/ua/news/all/odnogo-nebanki...
...,...,...
95,Страхового брокера включено до реєстру,https://bank.gov.ua/ua/news/all/strahovogo-bro...
96,Унесено зміни до Правил складання та подання з...,https://bank.gov.ua/ua/news/all/uneseno-zmini-...
97,Коментар Національного банку щодо рівня інфляц...,https://bank.gov.ua/ua/news/all/komentar-natsi...
98,Виступ Голови Національного банку Андрія Пишно...,https://bank.gov.ua/ua/news/all/vistup-golovi-...


In [84]:
import pandas as pd
import urllib
from sqlalchemy import create_engine, text

params = "Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=News_NBU;Trusted_Connection=yes;"
quoted = urllib.parse.quote_plus(params)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted}")

conn = engine.raw_connection() 
cursor = conn.cursor()

try:
    columns = ",".join([f"[{col}]" for col in df.columns])
    placeholders = ",".join(["?"] * len(df.columns))
    sql_query = f"INSERT INTO dbo.tb_news_main ({columns}) VALUES ({placeholders})"

    data_to_insert = df.values.tolist()

    cursor.fast_executemany = True 
    cursor.executemany(sql_query, data_to_insert)
    conn.commit()
    print("Data inserted successfully.")

except Exception as e:
    conn.rollback()
    print(f"Error: {e}")

finally:
    cursor.close()
    conn.close()

Data inserted successfully.


In [2]:
from google import genai
GEMINI_API_KEY = 'key'

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client(api_key = GEMINI_API_KEY)

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Проаналізуй новину https://bank.gov.ua/ua/news/all/aktualizovano-vimogi-do-strukturi-vlasnosti-nadavachiv-finansovih-poslug. Чи є там інформація про штрафи або інші заходи впливу від НБУ до банків? Якщо є, зроби таблицю з назвою банко, сумою штрафу (0 якщо це не штраф а інші обмеження) та описом підстав + дата"
)
print(response.text)

Проаналізувавши новину за посиланням https://bank.gov.ua/ua/news/all/aktualizovano-vimogi-do-strukturi-vlasnosti-nadavachiv-finansovih-poslug, можна зробити наступні висновки:

**Чи є там інформація про штрафи або інші заходи впливу від НБУ до банків?**

**Ні, у цій конкретній новині немає інформації про штрафи або інші заходи впливу від НБУ до банків чи інших фінансових установ.**

Ця новина є **регуляторним оголошенням** Національного банку України. Вона інформує про:

*   **Актуалізацію вимог** до структури власності надавачів фінансових послуг (як банків, так і небанківських установ).
*   **Приведення у відповідність** вимог для небанківських фінансових установ до вимог, що вже діють для банків.
*   **Уточнення критеріїв** бездоганної ділової репутації, значної участі, порядку доведення джерел коштів.
*   **Мету цих змін:** запобігання відмиванню коштів, забезпечення фінансової стабільності, захист прав споживачів.

Це повідомлення про **зміни в правилах та нормах**, які регулювати

In [3]:
from google import genai
GEMINI_API_KEY = 'key'

client = genai.Client(api_key = GEMINI_API_KEY)

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Проаналізуй новину https://bank.gov.ua/ua/news/all/finansoviy-kompaniyi-vidmovleno-u-prodovjenni-stroku-vikonannya-rishennya-natsionalnogo-banku-20215. Чи є там інформація про штрафи або інші заходи впливу від НБУ до банків? Якщо є, зроби таблицю з назвою банко, сумою штрафу (0 якщо це не штраф а інші обмеження) та описом підстав + дата. Відповідай коротко: просто слово Ні, якщо там немає потрібних даних, або таблиця, якщо є. І більше нічого"
)
print(response.text)

Ні


In [4]:
from google import genai
GEMINI_API_KEY = 'key'

client = genai.Client(api_key = GEMINI_API_KEY)

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Проаналізуй новину https://bank.gov.ua/ua/news/all/natsionalniy-bank-u-lyutomu-2025-roku-zastosuvav-do-pyati-bankiv-ta-shesti-nebankivskih-finansovih-ustanov-zahodi-vplivu-za-porushennya-u-sferi-finmonitoringu-ta-valyutnogo-zakonodavstva. Чи є там інформація про штрафи або інші заходи впливу від НБУ до банків? Якщо є, зроби таблицю з назвою банко, сумою штрафу (0 якщо це не штраф а інші обмеження) та описом підстав + дата. Відповідай коротко: просто слово Ні, якщо там немає потрібних даних, або таблиця, якщо є. І більше нічого"
)
print(response.text)

| Назва банку | Сума штрафу | Опис підстав | Дата |
|---|---|---|---|
| АТ "ПЕРШИЙ УКРАЇНСЬКИЙ МІЖНАРОДНИЙ БАНК" (ПУМБ) | 0 | За неналежне виконання обов’язку щодо здійснення поглибленої перевірки клієнта; неналежне виконання обов’язку щодо повідомлення спеціально уповноваженого органу про порогові фінансові операції; неналежне виконання обов’язку щодо забезпечення виявлення фінансових операцій, що підлягають фінансовому моніторингу, їх реєстрації та зберігання інформації про них; неналежне застосування у діяльності банку ризик-орієнтованого підходу. | Лютий 2024 |
| АТ "СЕНС БАНК" | 0 | За неналежне виконання обов’язку щодо управління ризиками, притаманними його діяльності, з метою виявлення, оцінки, моніторингу, контролю, зниження та внутрішнього звітування про такі ризики. | Лютий 2024 |
| АТ "УКРАЇНСЬКИЙ БУДІВЕЛЬНО-ІНВЕСТИЦІЙНИЙ БАНК" | 0 | За неналежне виконання обов’язку щодо вивчення клієнтів (неналежне застосування ризик-орієнтованого підходу); неналежне виконання банком обов’я

In [15]:
from google import genai
GEMINI_API_KEY = 'key'

client = genai.Client(api_key = GEMINI_API_KEY)

response = client.models.generate_content(
    model="gemini-3-flash-preview", contents="Проаналізуй новину https://bank.gov.ua/ua/news/all/natsionalniy-bank-u-lyutomu-2025-roku-zastosuvav-do-pyati-bankiv-ta-shesti-nebankivskih-finansovih-ustanov-zahodi-vplivu-za-porushennya-u-sferi-finmonitoringu-ta-valyutnogo-zakonodavstva. Чи є там інформація про штрафи або інші заходи впливу від НБУ до банків? Якщо є, зроби таблицю з назвою банко, сумою штрафу (0 якщо це не штраф а інші обмеження) та описом підстав + дата. Відповідай коротко: просто слово Ні, якщо там немає потрібних даних, або таблиця, якщо є. І більше нічого"
)
print(response.text)

| Назва банку | Сума штрафу (грн) | Опис підстав | Дата |
| :--- | :--- | :--- | :--- |
| АТ "АБ "РАДАБАНК" | 851 000 | Порушення у сфері фінмоніторингу: неналежна перевірка клієнтів (зокрема PEP), недоліки в системі управління ризиками та автоматизації. | 20.02.2025 |
| АТ "КБ "ГЛОБУС" | 500 000 | Порушення валютного законодавства щодо порядку надання інформації про валютні операції. | 24.02.2025 |
| АТ "АСВІО БАНК" | 400 000 | Неналежне функціонування системи внутрішнього контролю у сфері ПВК/ФТ. | 11.02.2025 |
| АТ "КРИСТАЛБАНК" | 51 000 | Порушення строків подання інформації про валютні операції. | 12.02.2025 |
| АТ "КОМІНВЕСТБАНК" | 0 | Письмове застереження за порушення у сфері фінмоніторингу щодо виконання обов’язків суб’єкта первинного фінмоніторингу. | 20.02.2025 |
